Notebook purpose

- Explore differences in savings behaviour


todo:

- Inflows into savings accounts

- ISA payments

- Investment and pension payments

In [1]:
import os
import sys

import linearmodels as lm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.txn_classifications as tc
import entropy.figures as figs
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh
from entropy import config

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

## Load data

In [2]:
dfs, dfl = hd.read_samples(["777", "XX7"])
hd.inspect(dfl)

Time for read_sample    : 1.82 seconds
Time for read_sample    : 4.94 minutes
(16,074,957, 31)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac
0,1088993,2012-04-02,57,23.74,1982 01apr 12 the broadway fruiterer london gb,NaN,NaN,NaN,0.0,n8 9,2010-05-13,10k to 20k,1987.0,2016-10-30,273998,2015-08-15 21:37:00,natwest bank,current,2014-07-18,2017-10-23,True,NaN,NaN,NaN,NaN,NaN,u,201204,NaN,22542.600586,2.425209
1,1088994,2012-04-02,57,25.68,1982 01apr 12 waitrose 835 crouch end gb,waitrose,spend,household,0.0,n8 9,2010-05-13,10k to 20k,1987.0,2016-10-30,273998,2015-08-15 21:37:00,natwest bank,current,2014-07-18,2017-08-15,True,NaN,waitrose,"food, groceries, household",NaN,supermarket,u,201204,NaN,22542.600586,2.425209


In [24]:
def add_income_pmt(df):
    mask = df.tag_group.eq("income")
    df["income_pmt"] = df.amount.where(mask, np.nan) * -1
    return df


dfl = add_income_pmt(dfl)
dfs = add_income_pmt(dfs)

## Timings of income payments and transfers to savings accounts

Income payments per month

In [20]:
def income_pmts_per_month(df):
    df = df.copy()
    return (
        df.groupby(["user_id", "ym"])
        .income_pmt.agg([("txns", "count"), ("income", "sum")])
        .groupby("user_id")
        .agg(
            [
                "mean",
                "median",
                "min",
                "max",
                "std",
                "size",
            ]
        )
    )


income_pmts_per_month(dfl).describe().T

count         mean          std          min          25%  \
txns   mean    5677.0     2.506022     2.073624     0.666667     1.133333   
       median  5677.0     2.426986     2.100056     1.000000     1.000000   
       min     5677.0     1.231108     1.608797     0.000000     0.000000   
       max     5677.0     4.350537     3.355715     1.000000     2.000000   
       std     5677.0     0.861687     0.743961     0.000000     0.432760   
       size    5677.0    28.873525    16.686351     6.000000    15.000000   
income mean    5677.0  2233.210205  1407.756592   292.851654  1316.344727   
       median  5677.0  2176.336670  1347.301147     0.015000  1319.520020   
       min     5677.0   893.116882  1158.431152 -4010.510010     0.000000   
       max     5677.0  4872.775879  5928.009277   425.399994  2171.280029   
       std     5677.0   935.188911  1158.416958     0.000000   323.420319   
       size    5677.0    28.873525    16.686351     6.000000    15.000000   

                       50%          75%           max  
txns   mean       1.921053     3.000000     19.090909  
       median     2.000000     3.000000     19.000000  
       min        1.000000     2.000000     18.000000  
       max        3.000000     5.000000     51.000000  
       std        0.687836     1.090500     11.998607  
       size      25.000000    38.000000    100.000000  
income mean    1872.958130  2728.284668  12378.085938  
       median  1859.750000  2687.479980  14122.665039  
       min      432.000000  1528.780029  10596.169922  
       max     3232.350098  5178.160156  94463.453125  
       std      639.714369  1102.914995  20987.941536  
       size      25.000000    38.000000    100.000000

Number of days between income payments

In [51]:
def days_since_last_income_pmt(df):
    return (
        df.set_index("date")
        .groupby("user_id")
        .resample("d")
        .income_pmt.max()
        .where(lambda x: x.isna(), 0)
        .where(lambda x: x.notna(), 1)
        # .cumsum()
    )


days_since_last_income_pmt(dfs)

user_id  date      
7777     2012-11-01    1.0
         2012-11-02    1.0
         2012-11-03    1.0
         2012-11-04    1.0
         2012-11-05    1.0
                      ... 
582777   2018-12-27    1.0
         2018-12-28    1.0
         2018-12-29    1.0
         2018-12-30    1.0
         2018-12-31    1.0
Name: income_pmt, Length: 55239, dtype: float32

Number of days between savings account inflows

Number of days since last income payment at time of savings account inflow